<a href="https://colab.research.google.com/github/zhannatoleubek-png/special-okx-chainsaw/blob/main/%D1%81%D1%82%D1%80%D0%BE%D0%B3%D0%B8%D0%B9%20%D0%B0%D0%BF%20%D1%83%D0%BF%D1%80%D0%BE%D1%89%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Colab-ready OKX Scanner with Enhanced Trend Detection

!pip install --quiet ccxt pandas numpy requests ta

import time
import datetime as dt
import requests
import pandas as pd
import numpy as np
import ccxt
from ta.momentum import RSIIndicator
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import EMAIndicator

# ---------------- CONFIG ----------------
CONFIG = {
    "telegram_bot_token": "YOUR_TELEGRAM_BOT_TOKEN",
    "telegram_chat_id": "YOUR_CHAT_ID",
    "scan_interval": 60,
    "min_volume_usdt": 100000,
}

# ---------------- TELEGRAM ----------------
def send_telegram(msg: str):
    try:
        requests.post(
            f"https://api.telegram.org/bot{CONFIG['telegram_bot_token']}/sendMessage",
            data={"chat_id": CONFIG["telegram_chat_id"], "text": msg, "parse_mode": "Markdown"}
        )
    except Exception as e:
        print(f"Telegram error: {e}")

# ---------------- EXCHANGE ----------------
exchange = ccxt.okx({"enableRateLimit": True})

def safe_fetch_ohlcv(symbol, timeframe='15m', limit=200):
    try:
        return exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    except Exception:
        return None

# ---------------- INDICATOR BLOCK ----------------
def calc_indicators(df):
    df["rsi"] = RSIIndicator(df["close"], 14).rsi()
    df["ema20"] = EMAIndicator(df["close"], 20).ema_indicator()
    df["ema50"] = EMAIndicator(df["close"], 50).ema_indicator()
    df["ema200"] = EMAIndicator(df["close"], 200).ema_indicator()
    df["obv"] = OnBalanceVolumeIndicator(df["close"], df["vol"]).on_balance_volume()
    return df

# ---------------- ENHANCED CONTEXT ANALYSIS ----------------
def context_analysis(symbol):
    """Context (1H–4H): adaptive trend detection with more tolerance"""
    for tf in ['1h', '4h']:
        ohlcv = safe_fetch_ohlcv(symbol, tf, limit=200)
        if not ohlcv:
            continue
        df = pd.DataFrame(ohlcv, columns=['ts','open','high','low','close','vol']).astype(float)
        ema50 = EMAIndicator(df['close'], 50).ema_indicator()
        ema200 = EMAIndicator(df['close'], 200).ema_indicator()
        last = df.iloc[-2]

        ema_slope = ema50.iloc[-2] - ema50.iloc[-7]
        price = last['close']
        ema_ratio = (ema50.iloc[-2] / ema200.iloc[-2]) - 1  # насколько EMA50 выше/ниже EMA200

        # 🔧 Более гибкий фильтр тренда:
        if (ema_ratio > 0.002 or ema_slope > 0):   # 0.2% и выше — ап
            return "up"
        elif (ema_ratio < -0.002 or ema_slope < 0):  # 0.2% и ниже — даун
            return "down"

    return "sideways"

# ---------------- SIGNAL SCANNER ----------------
def analyze_symbol(symbol):
    ohlcv = safe_fetch_ohlcv(symbol, '15m')
    if not ohlcv:
        return None

    df = pd.DataFrame(ohlcv, columns=['ts','open','high','low','close','vol']).astype(float)
    df = calc_indicators(df)
    last = df.iloc[-2]

    context = context_analysis(symbol)
    if context == "sideways":
        print(f"⛔ {symbol}: контекст sideways — пропуск")
        return None

    signal = None
    if context == "up":
        if last["rsi"] < 40 and last["close"] > last["ema50"] and last["ema50"] > last["ema200"]:
            signal = f"🟢 LONG {symbol} | RSI={last['rsi']:.1f}"
    elif context == "down":
        if last["rsi"] > 60 and last["close"] < last["ema50"] and last["ema50"] < last["ema200"]:
            signal = f"🔴 SHORT {symbol} | RSI={last['rsi']:.1f}"

    if signal:
        print(f"✅ {signal}")
        send_telegram(signal)
    else:
        print(f"⛔ {symbol}: нет сигнала")

# ---------------- SCANNER MAIN LOOP ----------------
def run_scanner():
    print("🚀 Scanner started...")
    markets = exchange.load_markets()
    usdt_pairs = [s for s in markets if "/USDT" in s]
    print(f"✅ Found {len(usdt_pairs)} liquid symbols (>100000 USDT 24h volume)")

    while True:
        for symbol in usdt_pairs:
            try:
                ticker = exchange.fetch_ticker(symbol)
                if ticker['quoteVolume'] and ticker['quoteVolume'] > CONFIG['min_volume_usdt']:
                    print(f"🔍 {symbol}")
                    analyze_symbol(symbol)
                else:
                    print(f"⏩ {symbol}: low volume")
            except Exception as e:
                print(f"⚠️ Error {symbol}: {e}")
                continue

        print(f"⏱ Waiting {CONFIG['scan_interval']}s before next scan...\n")
        time.sleep(CONFIG["scan_interval"])

# ---------------- START ----------------
run_scanner()


Выходные данные были обрезаны до нескольких последних строк (5000).
⏩ DOGE/USDT:USDT: low volume
⏩ XRP/USDT:USDT: low volume
⏩ BCH/USDT:USDT: low volume
⏩ PUMP/USDT:USDT: low volume
⏩ WLFI/USDT:USDT: low volume
⏩ LINEA/USDT:USDT: low volume
⏩ 0G/USDT:USDT: low volume
⏩ 1INCH/USDT:USDT: low volume
⏩ 2Z/USDT:USDT: low volume
⏩ A/USDT:USDT: low volume
⏩ AAVE/USDT:USDT: low volume
⏩ ACE/USDT:USDT: low volume
⏩ ACH/USDT:USDT: low volume
⏩ ACT/USDT:USDT: low volume
⏩ ADA/USDT:USDT: low volume
⏩ AERO/USDT:USDT: low volume
⏩ AEVO/USDT:USDT: low volume
⏩ AGLD/USDT:USDT: low volume
⏩ AI16Z/USDT:USDT: low volume
⏩ AIXBT/USDT:USDT: low volume
⏩ ALGO/USDT:USDT: low volume
⏩ ANIME/USDT:USDT: low volume
⏩ APE/USDT:USDT: low volume
⏩ API3/USDT:USDT: low volume
⏩ APT/USDT:USDT: low volume
⏩ AR/USDT:USDT: low volume
⏩ ARB/USDT:USDT: low volume
⏩ ARKM/USDT:USDT: low volume
⏩ ASTER/USDT:USDT: low volume
⏩ ATH/USDT:USDT: low volume
⏩ ATOM/USDT:USDT: low volume
⏩ AUCTION/USDT:USDT: low volume
⏩ AVAX/USDT:US